In [36]:
import socket
from socket import timeout
import time
import os

In [7]:
BUFFER_SIZE = 20

In [19]:
serverName = 'localhost'

PortRecv = 15001
PortSend = 15000

recv = (serverName, PortRecv)
send = (serverName, PortSend)

recv_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
send_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

recv_sock.bind(recv)

recv_sock.settimeout(5)

In [9]:
seq = 0
content = open('a.txt', "rb")

In [40]:
def process_ACK(message):
    if message == "":
        return  []
    acks = ([int(x) for x in message.split(",")])
    acks.sort()
    return acks

# 2 modes, create and update
# In update, first extra argument would be the ACK array from the server
#            second extra argument would be the last positive ack
def update_queue(arr=[], mode="create", N=100, neg_acks = [], last_ack=-1):
    global seq
    global content
    if mode=="create":
        seq = 0
        data = content.read(BUFFER_SIZE)
        while N>0 and data != b'':
            arr += [(seq, data)]
            seq += 1
            N -= 1
            # Prevent reading extra bits
            if N>0:
                data = content.read(BUFFER_SIZE)
    elif mode=="update":
        seq = arr[-1][0] + 1
        index = 0
        ack_index = 0
        
        # Removing successfully transmitted packets
        while index<len(arr) and arr[index][0]<=last_ack:
            print("index, ack_index, len(neg_acks), len(arr):{},{},{},{}".format(index, ack_index, len(neg_acks), len(arr)))
            if ack_index==len(neg_acks):
                print("Element:{} popped".format(arr[index][0]))
                arr.pop(index)
            elif neg_acks[ack_index] > arr[index][0]:
                print("element:{} popped".format(arr[index][0]))
                arr.pop(index)
            else:
                print("Element:{} skipped".format(arr[index][0]))
                index += 1
                ack_index += 1
        
        # Adding new packets in new empty spaces
        extra_space = N-len(arr)
        if extra_space>0:
            data = content.read(BUFFER_SIZE)
            while extra_space>0 and len(data) >0:
                arr += [(seq, data)]
                seq += 1
                extra_space -= 1
                if extra_space>0:
                    data = content.read(BUFFER_SIZE)
    return arr

In [41]:
def make_packet(seq,ack,data):
    seq_field = (seq).to_bytes(4,byteorder='big')
    ack_field = (ack).to_bytes(4,byteorder='big')
    packet =  seq_field + ack_field + data
    return packet

def upld(file_name):
    global seq
    global content
    seq = 0
    content = open(file_name, "rb")
    finished = os.path.getsize(file_name)//BUFFER_SIZE
    N = 3
    last_ack = -1
    current_queue = update_queue(N=N)
    print(current_queue)
    while current_queue:
        for part in current_queue:
            packet = make_packet(part[0], finished, part[1])
            send_sock.sendto(packet,send)
        print("Sent next Queue")
        while True:
            try:
                response ,address = recv_sock.recvfrom(4096)
                if last_ack < int.from_bytes(response[4:8],byteorder='big'):
                    break
            except timeout:
                break
        
        #seq = int.from_bytes(packet[:4],byteorder='big')
        #ack = int.from_bytes(packet[4:8],byteorder='big')
        #data = packet[8:].decode()
        
        negative_acks = process_ACK(response[8:].decode()) #******************** Edit the slicing according to packet
        last_ack = int.from_bytes(response[4:8],byteorder='big')  # Last positive ack is stored here
        current_queue = update_queue(current_queue, mode="update", N=N, neg_acks = negative_acks, last_ack=last_ack)
        print(current_queue)
        print(finished)
    # record throughput and time etc
    content.close()
    return

In [42]:
upld("a.txt")

[(0, b'I heard a glass shat'), (1, b'ter on the wall in t'), (2, b'he apartment above m')]
Sent next Queue
index, ack_index, len(neg_acks), len(arr):0,0,0,3
Element:0 popped
index, ack_index, len(neg_acks), len(arr):0,0,0,2
Element:1 popped
index, ack_index, len(neg_acks), len(arr):0,0,0,1
Element:2 popped
[(3, b'ine\r\nAt first I thou'), (4, b'ght that I was dream'), (5, b"in'\r\nBut then I hear")]
122
Sent next Queue
index, ack_index, len(neg_acks), len(arr):0,0,0,3
Element:3 popped
index, ack_index, len(neg_acks), len(arr):0,0,0,2
Element:4 popped
index, ack_index, len(neg_acks), len(arr):0,0,0,1
Element:5 popped
[(6, b'd the voice of a gir'), (7, b'l\r\nAnd it sounded li'), (8, b"ke she'd been cryin'")]
122
Sent next Queue
index, ack_index, len(neg_acks), len(arr):0,0,0,3
Element:6 popped
index, ack_index, len(neg_acks), len(arr):0,0,0,2
Element:7 popped
index, ack_index, len(neg_acks), len(arr):0,0,0,1
Element:8 popped
[(9, b"\r\nNow I'm too worrie"), (10, b"d to be sleepin'\r\nSo

index, ack_index, len(neg_acks), len(arr):0,0,0,3
Element:78 popped
index, ack_index, len(neg_acks), len(arr):0,0,0,2
Element:79 popped
index, ack_index, len(neg_acks), len(arr):0,0,0,1
Element:80 popped
[(81, b'e been the wind\r\nMus'), (82, b't have been the wind'), (83, b', it must have been ')]
122
Sent next Queue
index, ack_index, len(neg_acks), len(arr):0,0,0,3
Element:81 popped
index, ack_index, len(neg_acks), len(arr):0,0,0,2
Element:82 popped
index, ack_index, len(neg_acks), len(arr):0,0,0,1
Element:83 popped
[(84, b'the wind"\r\nAim my bo'), (85, b'ombox at the roof, I'), (86, b'\'m playing "Lean on ')]
122
Sent next Queue
index, ack_index, len(neg_acks), len(arr):0,0,0,3
Element:84 popped
index, ack_index, len(neg_acks), len(arr):0,0,0,2
Element:85 popped
index, ack_index, len(neg_acks), len(arr):0,0,0,1
Element:86 popped
[(87, b'Me"\r\nJust so that sh'), (88, b'e knows that she can'), (89, b' lean on me\r\nAnd whe')]
122
Sent next Queue
index, ack_index, len(neg_acks), len(

In [45]:
# Cell used for testing above function

arr = update_queue(N=30)
print("Initial arr:{}".format(arr))
message = process_ACK("1,4,3")
print(message)
arr=  update_queue(arr = arr, mode="update", N = 6, neg_acks = message, last_ack = 20)
print(arr)

Initial arr:[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f'), (6, 'g'), (7, 'h'), (8, 'i'), (9, 'j'), (10, 'k'), (11, 'l'), (12, 'm'), (13, 'n'), (14, 'o'), (15, 'p'), (16, 'q'), (17, 'r'), (18, 's'), (19, 't'), (20, 'u'), (21, 'v'), (22, 'w'), (23, 'x'), (24, 'y'), (25, 'z')]
[1, 3, 4]
index, ack_index, len(neg_acks), len(arr):0,0,3,26
element:(0, 'a') popped
index, ack_index, len(neg_acks), len(arr):0,0,3,25
Element:(1, 'b') skipped
index, ack_index, len(neg_acks), len(arr):1,1,3,25
element:(2, 'c') popped
index, ack_index, len(neg_acks), len(arr):1,1,3,24
Element:(3, 'd') skipped
index, ack_index, len(neg_acks), len(arr):2,2,3,24
Element:(4, 'e') skipped
index, ack_index, len(neg_acks), len(arr):3,3,3,24
Element:(5, 'f') popped
index, ack_index, len(neg_acks), len(arr):3,3,3,23
Element:(6, 'g') popped
index, ack_index, len(neg_acks), len(arr):3,3,3,22
Element:(7, 'h') popped
index, ack_index, len(neg_acks), len(arr):3,3,3,21
Element:(8, 'i') popped
index, ack_index, len(n

In [12]:
# Test cell
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

sock.bind(('localhost', 20000))

#sock.settimeout(1)

print(time.time())
for i in range(10):
    try:
        sock.recvfrom(1024)
        print("got here")
    except timeout:
        print("a")
print("reached here")

1617278279.0161037
a
a
a
a
a
a
a
a
a
a
reached here
